# Home Energy Management

A Home Energy Management System (HEMS) is designed to monitor, control, and optimize a household's energy source and usage. When a home has access to multiple energy supplies—such as grid electricity, solar power, battery storage, electric vehicle (EV), or backup generators—HEMS can intelligently decide which source to draw energy from to maximize efficiency, minimize costs, and reduce environmental impact.

A HEMS considers time-varying features of these energy sources, such as:

- Grid electricity: real-time pricing (RTP) of electricity (USD/kWh)
- Battery storage: battery capacity (kWh) and maximum input/output (kW)
- Electric vehicle: battery capacity (kWh), maximum input (kW), and ready time
- Solar power: variation of supply due to weather (sunshine hours)

In this example program, we optimize the energy supply mix of the above-mentioned sources using Fixstars Amplify. The present optimization aims to minimize energy costs. However, you can also add an objective to reduce CO2 emissions or other energy sources.

## Problem setting

To perform energy mix optimization, we need to provide the following information:

- Demand
  - Temporal variation of predicted total demands $\mathbf{d}$ (kW).
- Grid electricity
  - Temporal variation of predicted electricity price in RTP $\mathbf{p}$ (USD/kWh)
- Battery storage
  - Capacity of battery storage $c_{bs}$ (kWh)
  - Maximum input/output of battery storage $s_{bs, max}$ (kW)
- EV
  - EV-chargeable hours $\mathbf{b}$ (`bool`): Whether EV is parked at home (chargeable) at time $t$.
  - Capacity of EV battery $c_{be}$ (kWh)
  - Maximum charge input of EV battery $s_{be, max}$ (kW)
- Solar power
  - Temporal variation of predicted sunshine hours $\mathbf{h}$ (hours)
  - Solar panel efficiency
  - Solar irradiance (kW/m^2)
  - Panel area (m^2)

We will optimize for two days, which are discretized into 48 time slots (`num_slots`).

In [ ]:
import numpy as np

num_days = 2
num_slots = num_days * 24

# Demands
# Temporal variation of predicted total demands (kW)
demands = np.array([
    #[0am,  1am,  2am,  3am,  4am,  5am,  6am,  7am,  8am,  9am, 10am, 11am,
    #12pm, 13pm, 14pm, 15pm, 16pm, 17pm, 18pm, 19pm, 20pm, 21pm, 22pm, 23pm]
    [1.28, 0.82, 1.96, 0.44, 0.88, 1.82, 5.70, 7.40, 2.64, 2.72, 1.52, 2.10, # Day 1
     1.20, 1.26, 4.00, 1.24, 2.36, 5.90, 5.92, 6.82, 8.86, 4.78, 1.08, 1.40],
    [1.56, 1.02, 0.40, 1.26, 1.72, 1.72, 4.52, 7.32, 5.66, 1.26, 3.68, 3.20, # Day 2
     3.54, 3.96, 1.28, 3.94, 2.98, 8.26, 5.5 , 6.46, 4.76, 6.32, 1.98, 0.58],
    ])  # fmt: skip

# Grid electricity
# Temporal variation of predicted electricity price in RTP (USD/kWh)
grid_electricity_price = np.array([
    [0.15, 0.14, 0.14, 0.13, 0.13, 0.14, 0.17, 0.21, 0.25, 0.28, 0.30, 0.32, # Day 1
     0.34, 0.36, 0.37, 0.39, 0.40, 0.42, 0.41, 0.38, 0.35, 0.32, 0.29, 0.25],
    [0.16, 0.15, 0.15, 0.14, 0.14, 0.15, 0.18, 0.22, 0.26, 0.29, 0.31, 0.33, # Day 2
     0.35, 0.37, 0.38, 0.40, 0.41, 0.43, 0.42, 0.39, 0.36, 0.33, 0.30, 0.27],
    ])  # fmt: skip

# Battery storage
battery_storage_capacity = 40  # Capacity of battery storage (kWh)
battery_storage_max_input_output = 20  # Maximum input/output of battery storage (kW)

# EV
battery_ev_capacity = 50  # Capacity of EV battery (kWh)
battery_ev_max_input = 50  # Maximum charge input of EV battery (kW)
# EV-chargeable hours (whether EV is parked at home (chargeable) at time t).
ev_parked_at_home = np.array([
    [True, True, True, True, True, True, True, False, False, False, False, False,  # Day 1
     False, False, False, False, False, False, True, True, True, True, True, True, ],
    [True, True, True, True, True, True, True, False, False, False, False, False,  # Day 2
     False, False, False, False, False, False, True, True, True, True, True, True, ],
])  # fmt: skip

# Solar power
solar_panel_efficiency = 0.2  # Solar panel efficiency
solar_irradiance = 0.2  # Solar irradiance (kW/m^2)
panel_area = 20  # Panel area (m^2)
# Temporal variation of predicted sunshine hours (hours)
sunshine_hours = np.array([
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10, 0.30, 0.50, 0.70, 0.90, # Day 1
     1.00, 1.00, 0.90, 0.70, 0.50, 0.30, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.20, 0.40, 0.60, 0.80, 1.00, # Day 2
     0.90, 0.80, 0.70, 0.50, 0.30, 0.10, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
])  # hours  # fmt: skip

solar_power_max_supply = (
    solar_panel_efficiency * solar_irradiance * panel_area * sunshine_hours
)  # kW

assert len(np.ravel(demands)) == num_slots
assert len(np.ravel(grid_electricity_price)) == num_slots
assert len(np.ravel(ev_parked_at_home)) == num_slots
assert len(np.ravel(sunshine_hours)) == num_slots

Let us define a function `plot` and plot the problem setting. This function will be used to plot the solution obtained later as well.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook"


def plot(inp: dict[str, dict[str, np.ndarray]], show_ev_parked: bool = True):
    # Draw a vertical bar at every 24 hours
    def daily_bar(fig: go.Figure) -> None:
        for i in range(num_days + 1):
            fig.add_vline(x=i * 24, line_dash="dot", line_color="#555555")

    def get_ev_usage_period(bools: list | np.ndarray) -> tuple[list, list]:
        start: list[float] = []
        end: list[float] = []
        if bools[0]:
            start.append(0)
        for i in range(len(bools) - 1):
            if not bools[i] and bools[i + 1]:
                start.append(i + 0.5)
            elif bools[i] and not bools[i + 1]:
                end.append(i + 0.5)
        if bools[-1]:
            end.append(len(bools) - 1)
        assert len(start) == len(end), f"length does not match {start=}, {end=}"
        return start, end

    # Shade the time slots where EV is in use (not chargeable)
    def ev_usage(bools: list | np.ndarray, fig: go.Figure) -> None:
        start, end = get_ev_usage_period(bools)
        for s, e in zip(start, end):
            fig.add_vrect(
                x0=s,
                x1=e,
                fillcolor="black",
                opacity=0.1,
                annotation_text="EV in use",
                annotation_position="top left",
            )

    num_plots = len(inp.keys())

    fig = make_subplots(rows=num_plots, cols=1)

    for i in range(num_plots):
        label, data = list(inp.items())[i]
        for k, v in data.items():
            fig.add_trace(
                go.Scatter(
                    x=list(range(len(v))),
                    y=v,
                    mode="lines",
                    name=f"{k}",
                ),
                row=1 + i,
                col=1,
            )
        fig.update_yaxes(title_text=label, row=1 + i, col=1)

    daily_bar(fig)
    if show_ev_parked:
        ev_usage([not elem for elem in np.ravel(ev_parked_at_home)], fig)
    fig.update_xaxes(title_text="hours", row=num_plots, col=1)
    fig.show()


# Plot the problem setting
plot(
    {
        "demand (kW)": {"Total": np.ravel(demands)},
        "USD/kWh": {"grid electricity price": np.ravel(grid_electricity_price)},
        "supply (kW)": {"solar power": np.ravel(solar_power_max_supply)},
    },
)

## Formulation

### Decision variable

We will consider the following decision variables. For some decision variables, values are [pre-assigned](https://amplify.fixstars.com/en/docs/amplify/v1/constraint.html#fixing-variable-values).

- $\mathbf{q}_{ge}$ (`q_ge`): an integer variable array of a size `num_slots`. The value corresponds to the grid electricity supply (kW) whose bounds are (0, 30).

- $\mathbf{q}_{bs}$ (`q_bs`): an integer variable array of a size `num_slots`. The value corresponds to the remaining charge (kWh) in the battery. At the start and end of the optimization period, the battery is 50% charged.

- $\mathbf{q}_{be}$ (`q_be`): an integer variable array of a size `num_slots`. The value corresponds to the EV battery's remaining charge (kWh). At the start and end of the optimization period, the battery needs to be 50% charged. Also, the battery needs to be charged fully before scheduled use.

- $\mathbf{q}_{sp}$ (`q_sp`): a binary variable array of a size  `num_slots`. The value corresponds to whether the solar power is used or not.


In [ ]:
import amplify

gen = amplify.VariableGenerator()

# Grid electricity
# q_ge corresponds to the grid electricity supply: 0, 1, .., 30 (kW)
q_ge = gen.array("Integer", shape=num_slots, bounds=(0, 30))

# Battery storage
# q_bs corresponds to the remaining battery charge: 0, 1, 2, ..., battery_storage_capacity (kWh)
q_bs = gen.array("Integer", shape=num_slots, bounds=(0, battery_storage_capacity))
# boundary conditions (50% of remaining charge at start and end of the optimization period)
q_bs[0] = int(0.5 * battery_storage_capacity)
q_bs[-1] = int(0.5 * battery_storage_capacity)

# Battery EV
# q_be corresponds to the remaining battery charge: 0, 1, 2, ..., battery_ev_capacity (kWh)
q_be = gen.array("Integer", shape=num_slots, bounds=(0, battery_ev_capacity))
# pre-assign remaining batery charge to 10% if EV is not at home.
q_be = amplify.PolyArray(
    [q_be[i] if np.ravel(ev_parked_at_home)[i] else 0 for i in range(num_slots)]
)
# Boundary conditions
q_be[0] = int(0.5 * battery_ev_capacity)
q_be[-1] = int(0.5 * battery_ev_capacity)

# EV battery is fully charged when leaving house
for i in range(len(q_be) - 1):
    if np.ravel(ev_parked_at_home)[i] and not np.ravel(ev_parked_at_home)[i + 1]:
        q_be[i] = battery_ev_capacity

# Solar power
q_sp = gen.array("Binary", shape=num_slots)

### Supplies

The supply at time slot $t$ from each energy source can be formulated as follows:

- **Grid electricity (kW):**  
  $$s_{ge, t}=q_{ge, t}$$

- **Battery storage (kW):**  
  The battery's input/output (kW) is obtained based on the one-sided difference of the remaining charge (kWh) between $t$ and $t+1$, given that one time slot corresponds to one hour.  
  $$s_{bs, t} = -(q_{bs, t+1} - q_{bs,t})$$

- **EV battery (kW):**  
  The output from the EV battery is obtained in the same manner as battery storage. Additional multiplication by $b_{be, t+1}$ zeros any EV-related input/output since there is no EV battery contribution to household energy demands when EV is in use (`ev_parked_at_home` is `False`).  
  $$s_{be, t} = -(q_{be, t+1} - q_{be,t})\cdot b_{be, t+1}$$

- **Solar power (kW):**  
  $$s_{sp, t} = q_{sp, t} \cdot s_{sp, max, t}$$

Note that the negative supply is obtained when charging the battery.

In [ ]:
supply_ge = q_ge  # kW

supply_bs = -(q_bs.roll(-1) - q_bs)  # kW, assuming time slot is 1 hour

supply_be = -(
    (q_be.roll(-1) - q_be) * np.roll(np.ravel(ev_parked_at_home), -1)
)  # kW, assuming time slot is 1 hour, EV battery output (when EV is in use) is not for household demand so forcing zero supply.

supply_sp = q_sp * np.ravel(solar_power_max_supply)  # kW

### Constraints

The following "hard" constraints are considered.

- **Supply/demand:**  
  The total supply (kW) at any time slot $t$ must be greater than or equal to the total demand (kW).  
  
  $$d_t \le s_{ge,t} + s_{bs, t} + s_{be, t} + s_{sp, t}$$

  Note that the right-hand side is non-integer in general. Thus, `Relaxation` is chosen as the penalty formulation algorithm. See [documentation](https://amplify.fixstars.com/en/docs/amplify/v1/penalty.html#inequality-constraints) for details.

- **Battery storage:**  
  The input/output from/to battery storage must be less than or equal to $s_{bs, max}$ (kW).  
  
  $$s_{bs,t}^2 \le s_{bs,max}^2$$

  Note that the right-hand side is quadratic, meaning this inequality constraint may result in a fourth-order formulation. Thus, `Relaxation` is chosen as the penalty formulation algorithm. See [documentation](https://amplify.fixstars.com/en/docs/amplify/v1/penalty.html#inequality-constraints) for details.

- **Battery EV:**  
  The EV battery does not contribute to the household energy supply. Thus, the supply related to the EV battery is always zero or negative (charging). Also, the charge input does not exceed $s_{be, max}$ at any time slot $t$.  
  $$-s_{be, max} \le s_{be, t} \le 0$$

In [ ]:
constraint_demand = amplify.ConstraintList(
    [
        amplify.greater_equal(
            (supply_ge + supply_bs + supply_be + supply_sp)[t],
            np.ravel(demands)[t],
            penalty_formulation="Relaxation",
        )
        for t in range(num_slots)
    ]
)

constraint_bs = amplify.ConstraintList(
    [
        amplify.less_equal(
            supply_bs[t] * supply_bs[t],
            battery_storage_max_input_output * battery_storage_max_input_output,
            penalty_formulation="Relaxation",
        )
        for t in range(num_slots)
    ]
)

# EV does not do positive supply, only charing
constraint_be = amplify.ConstraintList(
    [amplify.clamp(supply_be[t], (-battery_ev_max_input, 0)) for t in range(num_slots)]
)

### Soft constraint

Usually, you may want to minimize switching between charging and discharging the battery storage. Such preference can be implemented using a hard constraint, but here, we consider this a "soft" constraint. A soft constraint is a part of an objective function where a penalty is added when the condition is violated. However, the solution with the constraint violated is still regarded as feasible, as the constraint is soft.

In this example program, we try to keep charging or discharging the battery storage for three hours. For this, the following penalty is added as a part of the objective function:

${\rm Minimize}\:f_{soft} = -\sum s_{bs, t} s_{bs,t + 1} -\sum s_{bs, t} s_{bs,t + 2} -\sum s_{bs, t} s_{bs,t + 3}$

With this soft constraint, the optimization would yield the same sign between $(s_{bs, t}, s_{bs,t + 1})$, $(s_{bs, t}, s_{bs,t + 2})$ and $(s_{bs, t}, s_{bs,t + 3})$, while enhancing the usage of the battery which can be achieved by larger $|s_{bs}|$.


In [ ]:
soft_constraint = (
    -(supply_bs * supply_bs.roll(-1))[:-1].sum()  # 1h
    - (supply_bs * supply_bs.roll(-2))[:-2].sum()  # 2h
    - (supply_bs * supply_bs.roll(-3))[:-3].sum()  # 3h
)

### Objectives

Finally, we construct the objective functions. There are two objectives (apart from the above soft constraint, which is technically an objective):

- Price
  - ${\rm Minimize}\:f_{price} = \sum s_{ge,t} \cdot p_t$
- Difference between total demand and supply
  - ${\rm Minimize} \:f_{diff} = \sum (s_{ge,t} + s_{bs,t} + s_{be,t} + s_{sp,t} - d_t)^2$

In [ ]:
objective_price = (supply_ge * np.ravel(grid_electricity_price)).sum()

supply_all = supply_ge + supply_bs + supply_be + supply_sp
objective_diff = (
    (np.ravel(demands) - supply_all) * (np.ravel(demands) - supply_all)
).sum()

## Execution

Now, let us solve the present optimization problem considering the hard constraints, soft constraints, and objective functions. Since we have relatively many terms to consider, finding good weights for objectives and constraints would be a tedious task. To make this more straightforward, we perform execution using the following two steps.

### Step 1. Execution for finding scaling factors

In the first step, we perform optimization only using hard constraints. The purpose is to find values for the objectives ($f_{diff}$, $f_{price}$ and $f_{soft}$), which are then used as scaling factors. 

Here, all objective functions, `objective_diff`, `objective_price`, and `soft_constraint` are the instance of `amplify.Poly` class, and we use the `evaluate` method of `amplify.Poly` to compute the objective values corresponding to the obtained best solution `result.best.values` (see [documentation](https://amplify.fixstars.com/en/docs/amplify/v1/evaluation.html#information-on-the-solutions) for details).

In [ ]:
from datetime import timedelta


def retriable_solve(
    model: amplify.Model, client: amplify.FixstarsClient
) -> amplify.Result:
    """A function that retries amplify.solve up to five times if no feasible solution is obtained."""
    for _ in range(5):
        result = amplify.solve(model, client)
        if len(result) > 0:
            break
        print("retrying amplify.solve...")
    if len(result) == 0:
        raise RuntimeError("no feasible solution found")
    return result


client = amplify.FixstarsClient()
# client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # If you use Amplify in a local environment, enter the Amplify API token.

# Constraint problem to obtain scaling factors
client.parameters.timeout = timedelta(milliseconds=5000)
# Obtain multiple solutions with the same objective value (there are multiple “optimal” solutions in Step 1, as no objective function is considered)
client.parameters.outputs.duplicate = True
model = amplify.Model(constraint_demand + constraint_bs + constraint_be)
result = retriable_solve(model, client)

# Calculate the objective function value for all feasible solutions, and consider
# the max value as the scaling factor for the corresponding objective function.
num_sols = len(result.solutions)
scaling_diff = np.array(
    [objective_diff.evaluate(result.solutions[i].values) for i in range(num_sols)]
).max()

scaling_price = np.array(
    [objective_price.evaluate(result.solutions[i].values) for i in range(num_sols)]
).max()

scaling_soft = np.abs(
    np.array(
        [soft_constraint.evaluate(result.solutions[i].values) for i in range(num_sols)]
    )
).max()

print(f"{scaling_diff=:.10e}")
print(f"{scaling_price=:.10e}")
print(f"{scaling_soft=:.10e}")

### Step 2. Execution for finding the solution

In the second step, we first divide $f_{diff}$, $f_{price}$ and $f_{soft}$ by the scaling factors obtained in step 1. With this procedure, all objectives are expected to yield values that are not too large or too small compared to the penalty value (=1) of the (penalized) hard constraints. Additional weights to the objectives may be added on top, so you can control which objectives to prioritize. Below, we use `1000`, `50`, and `0.1` for the scaled $f_{diff}$, $f_{price}$ and $f_{soft}$, respectively.

In [ ]:
# Actual solution with full constraints and objectives
client.parameters.timeout = timedelta(milliseconds=10000)
model = amplify.Model(
    1000 * objective_diff / scaling_diff
    + 50 * objective_price / scaling_price
    + 0.1 * soft_constraint / scaling_soft,
    constraint_demand + constraint_bs + constraint_be,
)

result = retriable_solve(model, client)

print(f"objective_diff: {objective_diff.evaluate(result.best.values):.2e}")
print(f"objective_price: {objective_price.evaluate(result.best.values):.2e}")
print(f"soft_constraint: {soft_constraint.evaluate(result.best.values):.2e}")

## Results

Let us plot the obtained energy mix using the function `plot` defined earlier.

In [ ]:
plot(
    {
        "suuply/demand (kW)": {
            "total demand": np.ravel(demands),
            "total supply": (supply_ge + supply_bs + supply_be + supply_sp).evaluate(
                result.best.values
            ),
            "grid power": supply_ge.evaluate(result.best.values),
            "battery storage": supply_bs.evaluate(result.best.values),
            "battery EV": supply_be.evaluate(result.best.values),
            "solar power (kW)": supply_sp.evaluate(result.best.values),
        },
        "battery level (kWh)": {
            "battery storage": q_bs.evaluate(result.best.values),
            "battery EV": q_be.evaluate(result.best.values),
        },
    },
)